<a href="https://colab.research.google.com/github/GrLau/Transformers-For-Stock-Price-Prediction/blob/main/Updated_Autoformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoformer Set Up

### First we grab the required code to set up the Autoformer
The original notebook code can be found [here](https://huggingface.co/blog/autoformer)

Hugging Face Autoformer Page [here](https://huggingface.co/docs/transformers/en/model_doc/autoformer#autoformer)

In [ ]:
prediction_length = 24
context_length = prediction_length*2
batch_size = 128
num_batches_per_epoch = 100
epochs = 50
scaling = "std"

In [ ]:
encoder_layers=2
decoder_layers=2
d_model=16

In [ ]:
!pip install -q transformers datasets evaluate accelerate "gluonts[torch]" ujson tqdm
!pip install -q protobuf --upgrade # without it, the evaluation code fails
!pip install datasets
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [ ]:
from gluonts.dataset.repository.datasets import get_dataset
from pprint import pprint


dataset = get_dataset("traffic")
freq = dataset.metadata.freq
prediction_length = dataset.metadata.prediction_length
print(f"traffic dataset prediction_length: {prediction_length}")

/usr/local/lib/python3.10/dist-packages/gluonts/dataset/repository/_lstnet.py:133: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_index = pd.period_range(


traffic dataset prediction_length: 24


In [ ]:
train_dataset = dataset.train
test_dataset = dataset.test

In [ ]:
from transformers import PretrainedConfig
from gluonts.time_feature import time_features_from_frequency_str

from gluonts.dataset.field_names import FieldName
from gluonts.transform import (
    AddAgeFeature,
    AddObservedValuesIndicator,
    AddTimeFeatures,
    AsNumpyArray,
    Chain,
    ExpectedNumInstanceSampler,
    RemoveFields,
    SelectFields,
    SetField,
    TestSplitSampler,
    Transformation,
    ValidationSplitSampler,
    VstackFeatures,
    RenameFields,
)

def create_transformation(freq: str, config: PretrainedConfig) -> Transformation:
    # create a list of fields to remove later
    remove_field_names = []
    if config.num_static_real_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_REAL)
    if config.num_dynamic_real_features == 0:
        remove_field_names.append(FieldName.FEAT_DYNAMIC_REAL)
    if config.num_static_categorical_features == 0:
        remove_field_names.append(FieldName.FEAT_STATIC_CAT)

    return Chain(
        # step 1: remove static/dynamic fields if not specified
        [RemoveFields(field_names=remove_field_names)]
        # step 2: convert the data to NumPy (potentially not needed)
        + (
            [
                AsNumpyArray(
                    field=FieldName.FEAT_STATIC_CAT,
                    expected_ndim=1,
                    dtype=int,
                )
            ]
            if config.num_static_categorical_features > 0
            else []
        )
        + (
            [
                AsNumpyArray(
                    field=FieldName.FEAT_STATIC_REAL,
                    expected_ndim=1,
                )
            ]
            if config.num_static_real_features > 0
            else []
        )
        + [
            AsNumpyArray(
                field=FieldName.TARGET,
                # we expect an extra dim for the multivariate case:
                expected_ndim=1 if config.input_size == 1 else 2,
            ),
            # step 3: handle the NaN's by filling in the target with zero
            # and return the mask (which is in the observed values)
            # true for observed values, false for nan's
            # the decoder uses this mask (no loss is incurred for unobserved values)
            # see loss_weights inside the xxxForPrediction model
            AddObservedValuesIndicator(
                target_field=FieldName.TARGET,
                output_field=FieldName.OBSERVED_VALUES,
            ),
            # step 4: add temporal features based on freq of the dataset
            # these serve as positional encodings
            AddTimeFeatures(
                start_field=FieldName.START,
                target_field=FieldName.TARGET,
                output_field=FieldName.FEAT_TIME,
                time_features=time_features_from_frequency_str(freq),
                pred_length=config.prediction_length,
            ),
            # step 5: add another temporal feature (just a single number)
            # tells the model where in the life the value of the time series is
            # sort of running counter
            AddAgeFeature(
                target_field=FieldName.TARGET,
                output_field=FieldName.FEAT_AGE,
                pred_length=config.prediction_length,
                log_scale=True,
            ),
            # step 6: vertically stack all the temporal features into the key FEAT_TIME
            VstackFeatures(
                output_field=FieldName.FEAT_TIME,
                input_fields=[FieldName.FEAT_TIME, FieldName.FEAT_AGE]
                + (
                    [FieldName.FEAT_DYNAMIC_REAL]
                    if config.num_dynamic_real_features > 0
                    else []
                ),
            ),
            # step 7: rename to match HuggingFace names
            RenameFields(
                mapping={
                    FieldName.FEAT_STATIC_CAT: "static_categorical_features",
                    FieldName.FEAT_STATIC_REAL: "static_real_features",
                    FieldName.FEAT_TIME: "time_features",
                    FieldName.TARGET: "values",
                    FieldName.OBSERVED_VALUES: "observed_mask",
                }
            ),
        ]
    )

In [ ]:
from gluonts.transform import InstanceSplitter
from gluonts.transform.sampler import InstanceSampler
from typing import Optional


def create_instance_splitter(
    config: PretrainedConfig,
    mode: str,
    train_sampler: Optional[InstanceSampler] = None,
    validation_sampler: Optional[InstanceSampler] = None,
) -> Transformation:
    assert mode in ["train", "validation", "test"]

    instance_sampler = {
        "train": train_sampler
        or ExpectedNumInstanceSampler(
            num_instances=1.0, min_future=config.prediction_length
        ),
        "validation": validation_sampler
        or ValidationSplitSampler(min_future=config.prediction_length),
        "test": TestSplitSampler(),
    }[mode]

    return InstanceSplitter(
        target_field="values",
        is_pad_field=FieldName.IS_PAD,
        start_field=FieldName.START,
        forecast_start_field=FieldName.FORECAST_START,
        instance_sampler=instance_sampler,
        past_length=config.context_length + max(config.lags_sequence),
        future_length=config.prediction_length,
        time_series_fields=["time_features", "observed_mask"],
    )

In [ ]:
from typing import Iterable

import torch
from gluonts.itertools import Cyclic, Cached
from gluonts.dataset.loader import as_stacked_batches


def create_train_dataloader(
    config: PretrainedConfig,
    freq,
    data,
    batch_size: int,
    num_batches_per_epoch: int,
    shuffle_buffer_length: Optional[int] = None,
    cache_data: bool = True,
    **kwargs,
) -> Iterable:
    PREDICTION_INPUT_NAMES = [
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    TRAINING_INPUT_NAMES = PREDICTION_INPUT_NAMES + [
        "future_values",
        "future_observed_mask",
    ]

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(data, is_train=True)
    if cache_data:
        transformed_data = Cached(transformed_data)

    # we initialize a Training instance
    instance_splitter = create_instance_splitter(config, "train")

    # the instance splitter will sample a window of
    # context length + lags + prediction length (from the 366 possible transformed time series)
    # randomly from within the target time series and return an iterator.
    stream = Cyclic(transformed_data).stream()
    training_instances = instance_splitter.apply(stream)

    return as_stacked_batches(
        training_instances,
        batch_size=batch_size,
        shuffle_buffer_length=shuffle_buffer_length,
        field_names=TRAINING_INPUT_NAMES,
        output_type=torch.tensor,
        num_batches_per_epoch=num_batches_per_epoch,
    )

In [ ]:
def create_backtest_dataloader(
    config: PretrainedConfig,
    freq,
    data,
    batch_size: int,
    **kwargs,
):
    PREDICTION_INPUT_NAMES = [
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(data)

    # we create a Validation Instance splitter which will sample the very last
    # context window seen during training only for the encoder.
    instance_sampler = create_instance_splitter(config, "validation")

    # we apply the transformations in train mode
    testing_instances = instance_sampler.apply(transformed_data, is_train=True)

    return as_stacked_batches(
        testing_instances,
        batch_size=batch_size,
        output_type=torch.tensor,
        field_names=PREDICTION_INPUT_NAMES,
    )

In [ ]:
def create_test_dataloader(
    config: PretrainedConfig,
    freq,
    data,
    batch_size: int,
    **kwargs,
):
    PREDICTION_INPUT_NAMES = [
        "past_time_features",
        "past_values",
        "past_observed_mask",
        "future_time_features",
    ]
    if config.num_static_categorical_features > 0:
        PREDICTION_INPUT_NAMES.append("static_categorical_features")

    if config.num_static_real_features > 0:
        PREDICTION_INPUT_NAMES.append("static_real_features")

    transformation = create_transformation(freq, config)
    transformed_data = transformation.apply(data, is_train=False)

    # We create a test Instance splitter to sample the very last
    # context window from the dataset provided.
    instance_sampler = create_instance_splitter(config, "test")

    # We apply the transformations in test mode
    testing_instances = instance_sampler.apply(transformed_data, is_train=False)

    return as_stacked_batches(
        testing_instances,
        batch_size=batch_size,
        output_type=torch.tensor,
        field_names=PREDICTION_INPUT_NAMES,
    )

In [ ]:
from transformers import AutoformerConfig, AutoformerForPrediction

config = AutoformerConfig.from_pretrained("kashif/autoformer-traffic-hourly")
model = AutoformerForPrediction.from_pretrained("kashif/autoformer-traffic-hourly")

test_dataloader = create_backtest_dataloader(
    config=config,
    freq=freq,
    data=test_dataset,
    batch_size=64,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/116k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/gluonts/time_feature/_base.py:249: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)


### Prepare our stock data

In [ ]:
#Creating Apple dataset
from datasets import load_dataset
import pandas as pd
import torch

#Load in Apple data
apple_dataset = load_dataset("Ammok/apple_stock_price_from_1980-2021", split = "train")

#Create train-test splits
my_train_dataset = apple_dataset.select(range(8374))
my_test_dataset = apple_dataset.select(range(8374, 10468))

print(f"Number of rows in train_dataset: {len(my_train_dataset)}")
print(f"Number of rows in test_dataset: {len(my_test_dataset)}")


my_train_dataset = my_train_dataset.remove_columns(['Date'])
my_test_dataset = my_test_dataset.remove_columns(['Date'])

#Within train split, separate into past data and "future" data
past_split = my_train_dataset.select(range(6152))
future_split = my_train_dataset.select(range(6702, 8374))
#Skips over some values due to tensor reshaping restrictions
#model requires tensors to be size (num, 769, 5) for "features" and (num, 769) for "values"



past_feature_split = past_split.remove_columns(['Close'])
future_feature_split = future_split.remove_columns(['Close'])


past_value_split = past_split.remove_columns(['Open', 'High', 'Low', 'Adj Close', 'Volume'])


past_feature_df = pd.DataFrame(past_feature_split)
future_feature_df = pd.DataFrame(future_feature_split)
past_value_df = pd.DataFrame(past_value_split)


#Creating tensors to feed into outputs
past_feature_np = past_feature_df.to_numpy()
tensor = torch.tensor(past_feature_np, dtype=torch.float32)
apple_past_feature = tensor.reshape(8, 769, 5)
print(apple_past_feature.shape)

past_value_np = past_value_df.to_numpy()
tensor = torch.tensor(past_value_np, dtype=torch.float32)
apple_past_value = tensor.reshape(8, 769)
print(apple_past_value.shape)

future_feature_np = future_feature_df.to_numpy()
tensor = torch.tensor(future_feature_np, dtype=torch.float32)
apple_future_feature = tensor.reshape(8, 209, 5)
print(apple_future_feature.shape)

apple_past_mask = torch.ones(8, 769)


#Add to dictionary
apple_dictionary = {}

apple_dictionary["past_features"] = apple_past_feature
apple_dictionary["past_values1"] = apple_past_value
apple_dictionary["past_observed_mask1"] = apple_past_mask
apple_dictionary["future_features"] = apple_future_feature

Generating train split:   0%|          | 0/10468 [00:00<?, ? examples/s]

Number of rows in train_dataset: 8374
Number of rows in test_dataset: 2094
torch.Size([8, 769, 5])
torch.Size([8, 769])
torch.Size([8, 209, 5])


In [ ]:
#Creating batches to test with
new_test_dataset = my_test_dataset.remove_columns(['Open', 'High', 'Low', 'Adj Close', 'Volume'])
batch1 = new_test_dataset.select(range(0, 349))
batch2 = new_test_dataset.select(range(349, 698))
batch3 = new_test_dataset.select(range(698, 1047))
batch4 = new_test_dataset.select(range(1047, 1396))
batch5 = new_test_dataset.select(range(1396, 1745))
batch6 = new_test_dataset.select(range(1745, 2094))

#Transforming into DataFrames in order to turn them into numpy arrays
batch1_df = pd.DataFrame(batch1)
batch2_df = pd.DataFrame(batch2)
batch3_df = pd.DataFrame(batch3)
batch4_df = pd.DataFrame(batch4)
batch5_df = pd.DataFrame(batch5)
batch6_df = pd.DataFrame(batch6)

#Turn into numpy arrays to match evaluation function
batch1_np = batch1_df.to_numpy()
batch2_np = batch2_df.to_numpy()
batch3_np = batch3_df.to_numpy()
batch4_np = batch4_df.to_numpy()
batch5_np = batch5_df.to_numpy()
batch6_np = batch6_df.to_numpy()

#Reshape to fit evaluation function
batch1_np = batch1_np.reshape(-1)
batch2_np = batch2_np.reshape(-1)
batch3_np = batch3_np.reshape(-1)
batch4_np = batch4_np.reshape(-1)
batch5_np = batch5_np.reshape(-1)
batch6_np = batch6_np.reshape(-1)


#Add to dictionary
apple_test = [{"target": batch1_np}, {"target": batch2_np}, {"target": batch3_np}, {"target": batch4_np}, {"target": batch5_np}, {"target": batch6_np}]

In [ ]:
import yfinance as yf
import datetime
import pandas as pd
import torch

start = datetime.date(1997, 5, 15)
end = datetime.date(2024, 1, 1)
current_date = datetime.date(2024, 6, 14)
codelist = ["AMZN"]
amzn_train_dataset = yf.download(codelist, start=start, end=end, auto_adjust=False)
amzn_test_dataset = yf.download(codelist, start=end, end=current_date, auto_adjust=False)

# Reset the index to remove dates
amzn_train_dataset.reset_index(inplace=True)
amzn_test_dataset.reset_index(inplace=True)

# Drop the 'Date' column if you don't need it anymore
amzn_train_dataset.drop(columns=['Date'], inplace=True)
amzn_test_dataset.drop(columns=['Date'], inplace=True)

print()



amzn_past_split = amzn_train_dataset[0:4614]
amzn_future_split = amzn_train_dataset[5363:6701]


amzn_past_feature_split = amzn_past_split.drop(columns=['Close'])
amzn_future_feature_split = amzn_future_split.drop(columns=['Close'])


amzn_past_value_split = amzn_past_split.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])



amzn_past_feature_df = pd.DataFrame(amzn_past_feature_split)
amzn_future_feature_df = pd.DataFrame(amzn_future_feature_split)
amzn_past_value_df = pd.DataFrame(amzn_past_value_split)


#Creating tensors to feed into outputs
amzn_past_feature_np = amzn_past_feature_df.to_numpy()
tensor = torch.tensor(amzn_past_feature_np, dtype=torch.float32)
amzn_past_feature = tensor.reshape(6, 769, 5)
print(amzn_past_feature.shape)

amzn_past_value_np = amzn_past_value_df.to_numpy()
tensor = torch.tensor(amzn_past_value_np, dtype=torch.float32)
amzn_past_value = tensor.reshape(6, 769)
print(amzn_past_value.shape)

amzn_future_feature_np = amzn_future_feature_df.to_numpy()
tensor = torch.tensor(amzn_future_feature_np, dtype=torch.float32)
amzn_future_feature = tensor.reshape(6, 223, 5)
print(amzn_future_feature.shape)

amzn_past_mask = torch.ones(6, 769)


#Add to dictionary
amzn_dictionary = {}

amzn_dictionary["past_features"] = amzn_past_feature
amzn_dictionary["past_values1"] = amzn_past_value
amzn_dictionary["past_observed_mask1"] = amzn_past_mask
amzn_dictionary["future_features"] = amzn_future_feature

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


torch.Size([6, 769, 5])
torch.Size([6, 769])
torch.Size([6, 223, 5])


In [ ]:
#Creating batches to test with
new_amzn_test_dataset = amzn_test_dataset.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])
batch1 = new_amzn_test_dataset[0:114]

#Transforming into DataFrames in order to turn them into numpy arrays
batch1_df = pd.DataFrame(batch1)

#Turn into numpy arrays to match evaluation function
batch1_np = batch1_df.to_numpy()

#Reshape to fit evaluation function
batch1_np = batch1_np.reshape(-1)

#Add to dictionary
amzn_test = [{"target": batch1_np}]

### Next, we load our data and evaluate the performance

In [ ]:
from accelerate import Accelerator
from tqdm.autonotebook import tqdm

accelerator = Accelerator()
device = accelerator.device
model.to(device)
model.eval()


#count = 0
forecasts_ = []
for batch in tqdm(test_dataloader):
    outputs = model.generate(
        static_categorical_features=batch["static_categorical_features"].to(device)
        if config.num_static_categorical_features > 0
        else None,
        static_real_features=batch["static_real_features"].to(device)
        if config.num_static_real_features > 0
        else None,
        past_time_features=apple_dictionary["past_features"].to(device),
        past_values=apple_dictionary["past_values1"].to(device),
        future_time_features=apple_dictionary["future_features"].to(device),
        past_observed_mask=apple_dictionary["past_observed_mask1"].to(device),
    )

    forecasts_.append(outputs.sequences.cpu().numpy())

    #count += 1
    #if count > 10: Uncomment if you wish to reduce the amount of batches
        #break

0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/gluonts/dataset/common.py:263: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return pd.Period(val, freq)


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
import numpy as np

forecasts = np.vstack(forecasts_)
print(forecasts.shape)

In [ ]:
from tqdm.autonotebook import tqdm
from evaluate import load
from gluonts.time_feature import get_seasonality

mae_metric = load("evaluate-metric/mae")

forecast_median = np.median(forecasts, 1)

mae_metrics = []
for item_id, ts in enumerate(tqdm(apple_test)): #Change to amzn_test or test_dataset for traffic data
    training_data = ts["target"][:-prediction_length]
    ground_truth = ts["target"][-prediction_length:]


    mae = mae_metric.compute(
        predictions=forecast_median[item_id],
        references=np.array(ground_truth))
    mae_metrics.append(mae["mae"])

In [ ]:
print(f"Autoformer univariate MAE: {np.mean(mae_metrics):.3f}")